# Goal:
Generate NBA player comparisons for every 2020 NBA Draft prospect for our shiny app.

Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import getpass
import psycopg2
from sklearn.preprocessing import scale, LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import euclidean
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
sns.set_style('whitegrid')
sns.set(rc = {'figure.figsize':(12, 8)})

Connect to our database

In [2]:
mypasswd = getpass.getpass()
conn = psycopg2.connect(database = 'cs20_group4',
                              user = 'jpcp73',#replace with pawprint
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)
cursor = conn.cursor()

········


Select our 2020 NBA Draft prospects and include their collegiate demographic data along with their NBA career probability predictions.

In [3]:
data = pd.read_sql_query("""
SELECT
    d.ncaa_id
    , d.player_name
    , COALESCE(c.team, n.school_or_team) as school
    , COALESCE(c.height, n.height) as height
    , COALESCE(c.weight, n.weight) as weight
    , n.class
    , d.player_score
    , n.rank
    , d.predicted_career_role
    , d.star_player_prob
    , d.starter_player_prob
    , d.role_player_prob
    , d.benchwarmer_prob
    , d.reserve_player_prob
    , d.non_nba_prob
FROM draft_rankings d
LEFT JOIN ncaa_current_rosters c ON d.ncaa_id = c.player_id
LEFT JOIN nba_top100_prospect_rankings n ON d.player_name = n.player
""", con = conn)
data = data.sort_values(by = ['player_score', 'rank'], ascending = False).reset_index(drop = True)
print(data.shape)
data.head()

(5111, 15)


,ncaa_id,player_name,school,height,weight,class,player_score,rank,predicted_career_role,star_player_prob,starter_player_prob,role_player_prob,benchwarmer_prob,reserve_player_prob,non_nba_prob
0,anthony-edwards-2,Anthony Edwards,Georgia,77.0,225.0,Fr.,6.000000,1.0,Star,1.000000,9.108253e-11,5.600788e-09,1.687002e-08,7.561751e-13,1.086892e-19
1,jordan-nwora-1,Jordan Nwora,Louisville,79.0,225.0,Jr.,6.000000,22.0,Star,1.000000,2.386891e-14,9.667769e-10,4.325299e-15,8.124112e-09,4.532037e-26
2,james-wiseman-1,James Wiseman,Memphis,85.0,240.0,Fr.,5.999999,3.0,Star,0.999999,1.863512e-07,2.910505e-07,1.126775e-07,1.739439e-09,1.482664e-15
3,nfaly-dante-1,N'Faly Dante,Oregon,83.0,230.0,Fr.,5.999024,12.0,Star,0.999518,6.388787e-06,4.675889e-04,7.981438e-06,1.380591e-08,3.313045e-09
4,tre-jones-1,Tre Jones,Duke,75.0,185.0,So.,5.977293,43.0,Star,0.978988,1.931996e-02,1.691163e-03,0.000000e+00,1.734437e-07,6.708136e-07


In [4]:
data.head(60)

,ncaa_id,player_name,school,height,weight,class,player_score,rank,predicted_career_role,star_player_prob,starter_player_prob,role_player_prob,benchwarmer_prob,reserve_player_prob,non_nba_prob
0,anthony-edwards-2,Anthony Edwards,Georgia,77.0,225.0,Fr.,6.000000,1.0,Star,1.000000e+00,9.108253e-11,5.600788e-09,1.687002e-08,7.561751e-13,1.086892e-19
1,jordan-nwora-1,Jordan Nwora,Louisville,79.0,225.0,Jr.,6.000000,22.0,Star,1.000000e+00,2.386891e-14,9.667769e-10,4.325299e-15,8.124112e-09,4.532037e-26
2,james-wiseman-1,James Wiseman,Memphis,85.0,240.0,Fr.,5.999999,3.0,Star,9.999994e-01,1.863512e-07,2.910505e-07,1.126775e-07,1.739439e-09,1.482664e-15
3,nfaly-dante-1,N'Faly Dante,Oregon,83.0,230.0,Fr.,5.999024,12.0,Star,9.995176e-01,6.388787e-06,4.675889e-04,7.981438e-06,1.380591e-08,3.313045e-09
4,tre-jones-1,Tre Jones,Duke,75.0,185.0,So.,5.977293,43.0,Star,9.789881e-01,1.931996e-02,1.691163e-03,0.000000e+00,1.734437e-07,6.708136e-07
5,precious-achiuwa-1,Precious Achiuwa,Memphis,81.0,225.0,Fr.,5.953998,11.0,Star,9.833885e-01,3.342032e-03,1.425379e-05,2.732259e-03,1.045609e-02,1.652807e-08
6,josh-green-2,Josh Green,Arizona,78.0,210.0,Fr.,5.912573,24.0,Star,9.514367e-01,1.409322e-02,3.263250e-02,4.230150e-06,6.216014e-04,6.425193e-05
7,marcus-evans-2,Marcus Evans,Virginia Commonwealth,74.0,190.0,None,5.895392,NaN,Star,9.834157e-01,2.417574e-05,6.017246e-05,1.642899e-03,1.190170e-03,1.299353e-02
8,isaiah-stewart-2,Isaiah Stewart,Washington,81.0,250.0,Fr.,5.287022,15.0,Star,7.447757e-01,5.306410e-02,2.056187e-03,4.758287e-02,1.485925e-01,2.457558e-05
9,onyeka-okongwu-1,Onyeka Okongwu,Southern California,81.0,245.0,Fr.,5.214092,13.0,Star,6.200812e-01,1.749937e-01,4.136304e-03,2.732344e-04,2.005146e-01,2.266999e-12


Read in our binned player data for the 2020 NBA Draft class. 

In [5]:
active = pd.read_csv('../Data/active_binned.csv')
active = active.sort_values(by = 'high_school_national_rank').drop_duplicates(subset = ['ncaa_id'])
active['active'] = 1
active['made_nba'] = 0
print(active.shape)
active.head()

(5111, 403)


,ncaa_id,player_name,nba_career_quality,three_pointers_bin_1,three_pointers_bin_2,three_pointers_bin_3,three_pointers_bin_4,free_throw_percentage_bin_1,free_throw_percentage_bin_2,free_throw_percentage_bin_3,free_throw_attempt_rate_bin_1,free_throw_attempt_rate_bin_2,free_throw_attempt_rate_bin_3,free_throw_attempt_rate_bin_4,free_throw_attempt_rate_bin_5,free_throw_attempt_rate_bin_6,games_played_bin_1,games_played_bin_2,games_played_bin_3,defensive_win_shares_bin_1,defensive_win_shares_bin_2,defensive_win_shares_bin_3,defensive_win_shares_bin_4,defensive_rebounds_bin_1,defensive_rebounds_bin_2,defensive_rebounds_bin_3,defensive_rebounds_bin_4,turnover_percentage_bin_1,turnover_percentage_bin_2,turnover_percentage_bin_3,two_point_percentage_bin_1,two_point_percentage_bin_2,two_point_percentage_bin_3,two_point_percentage_bin_4,assist_percentage_bin_1,assist_percentage_bin_2,assist_percentage_bin_3,assist_percentage_bin_4,three_point_attempt_rate_bin_1,three_point_attempt_rate_bin_2,three_point_attempt_rate_bin_3,weight_bin_1,weight_bin_2,weight_bin_3,weight_bin_4,weight_bin_5,high_school_recruit_score_bin_2,high_school_recruit_score_bin_3,high_school_recruit_score_bin_4,high_school_recruit_score_bin_5,blocks_bin_1,blocks_bin_2,blocks_bin_3,blocks_bin_4,blocks_bin_5,games_started_bin_1,games_started_bin_2,games_started_bin_3,games_started_bin_4,offensive_win_shares_bin_1,offensive_win_shares_bin_2,offensive_win_shares_bin_3,assists_bin_1,assists_bin_2,assists_bin_3,assists_bin_4,two_point_attempts_bin_1,two_point_attempts_bin_2,two_point_attempts_bin_3,two_point_attempts_bin_4,points_bin_1,points_bin_2,points_bin_3,points_bin_4,offensive_rebounds_bin_1,offensive_rebounds_bin_2,offensive_rebounds_bin_3,offensive_rebounds_bin_4,free_throw_attempts_bin_1,free_throw_attempts_bin_2,free_throw_attempts_bin_3,free_throw_attempts_bin_4,two_pointers_bin_1,two_pointers_bin_2,two_pointers_bin_3,two_pointers_bin_4,three_point_attempts_bin_1,three_point_attempts_bin_2,three_point_attempts_bin_3,three_point_attempts_bin_4,draft_rank_bin_1,draft_rank_bin_2,draft_rank_bin_3,draft_rank_bin_4,draft_rank_bin_5,turnovers_bin_1,turnovers_bin_2,turnovers_bin_3,turnovers_bin_4,height_bin_1,height_bin_2,height_bin_3,height_bin_4,height_bin_5,height_bin_6,field_goal_percentage_bin_1,field_goal_percentage_bin_2,field_goal_percentage_bin_3,field_goal_percentage_bin_4,field_goal_percentage_bin_5,minutes_played_bin_1,minutes_played_bin_2,minutes_played_bin_3,minutes_played_bin_4,high_school_national_rank_bin_1,high_school_national_rank_bin_2,field_goal_attempts_bin_1,field_goal_attempts_bin_2,field_goal_attempts_bin_3,field_goal_attempts_bin_4,field_goals_bin_1,field_goals_bin_2,field_goals_bin_3,field_goals_bin_4,total_rebounds_bin_1,total_rebounds_bin_2,total_rebounds_bin_3,total_rebounds_bin_4,personal_fouls_bin_1,personal_fouls_bin_2,personal_fouls_bin_3,personal_fouls_bin_4,steals_bin_1,steals_bin_2,steals_bin_3,steals_bin_4,three_point_percentage_bin_1,three_point_percentage_bin_2,three_point_percentage_bin_3,three_point_percentage_bin_4,usage_percentage_bin_1,usage_percentage_bin_2,usage_percentage_bin_3,height_to_weight_bin_1,height_to_weight_bin_2,height_to_weight_bin_3,height_to_weight_bin_4,height_to_weight_bin_6,weight_to_height_bin_1,weight_to_height_bin_2,weight_to_height_bin_3,weight_to_height_bin_4,weight_to_height_bin_5,weight_to_height_bin_6,recruit_draft_rank_mean_bin_1,recruit_draft_rank_mean_bin_2,draft_recruit_ratio_bin_1,draft_recruit_ratio_bin_2,draft_recruit_ratio_bin_3,assist_to_turnover_ratio_bin_1,assist_to_turnover_ratio_bin_2,assist_to_turnover_ratio_bin_3,assist_to_turnover_ratio_bin_4,assist_to_turnover_ratio_bin_5,assist_to_turnover_ratio_bin_6,assists_per_minute_bin_1,assists_per_minute_bin_2,assists_per_minute_bin_3,assists_per_minute_bin_4,assists_per_minute_bin_5,assists_per_minute_bin_6,blocks_per_minute_bin_1,blocks_per_minute_bin_2,blocks_per_minute_bin_3,blocks_per_minute_bin_4,blocks_per_minute_bin_5,drb_per_minute

Read in our binned player data for the historical players data. This set will serve as our NBA player baseline to compare against the 2020 NBA Draft prospects.

In [6]:
binned = pd.read_csv('../Data/binned_data.csv')
binned = binned.sort_values(by = 'high_school_national_rank').drop_duplicates(subset = ['ncaa_id'])
binned['active'] = 0
print(binned.shape)
binned.head()

(22046, 420)


,ncaa_id,nba_id,player_name,currently_in_school,height_bin_1,height_bin_2,height_bin_3,height_bin_4,height_bin_5,weight_bin_1,weight_bin_2,weight_bin_3,weight_bin_4,weight_bin_5,high_school_national_rank_bin_1,high_school_national_rank_bin_2,high_school_recruit_score_bin_2,high_school_recruit_score_bin_3,high_school_recruit_score_bin_4,high_school_recruit_score_bin_5,high_school_recruit_score_bin_6,draft_rank_bin_1,draft_rank_bin_2,draft_rank_bin_3,draft_rank_bin_4,draft_rank_bin_5,age_bin_2,age_bin_3,age_bin_4,age_bin_5,assists_bin_1,assists_bin_2,assists_bin_3,assists_bin_4,blocks_bin_1,blocks_bin_2,blocks_bin_3,blocks_bin_4,blocks_bin_5,defensive_rebounds_bin_1,defensive_rebounds_bin_2,defensive_rebounds_bin_3,defensive_rebounds_bin_4,field_goal_attempts_bin_1,field_goal_attempts_bin_2,field_goal_attempts_bin_3,field_goal_attempts_bin_4,field_goals_bin_1,field_goals_bin_2,field_goals_bin_3,field_goals_bin_4,field_goal_percentage_bin_1,field_goal_percentage_bin_2,field_goal_percentage_bin_3,field_goal_percentage_bin_4,field_goal_percentage_bin_5,free_throw_attempt_rate_bin_1,free_throw_attempt_rate_bin_2,free_throw_attempt_rate_bin_3,free_throw_attempt_rate_bin_4,free_throw_attempt_rate_bin_5,free_throw_attempt_rate_bin_6,free_throw_attempts_bin_1,free_throw_attempts_bin_2,free_throw_attempts_bin_3,free_throw_attempts_bin_4,free_throw_percentage_bin_1,free_throw_percentage_bin_2,free_throw_percentage_bin_3,games_played_bin_1,games_played_bin_2,games_played_bin_3,games_started_bin_1,games_started_bin_2,games_started_bin_3,games_started_bin_4,minutes_played_bin_1,minutes_played_bin_2,minutes_played_bin_3,minutes_played_bin_4,offensive_rebounds_bin_1,offensive_rebounds_bin_2,offensive_rebounds_bin_3,offensive_rebounds_bin_4,personal_fouls_bin_1,personal_fouls_bin_2,personal_fouls_bin_3,personal_fouls_bin_4,points_bin_1,points_bin_2,points_bin_3,points_bin_4,steals_bin_1,steals_bin_2,steals_bin_3,steals_bin_4,three_point_attempt_rate_bin_1,three_point_attempt_rate_bin_2,three_point_attempt_rate_bin_3,three_point_attempts_bin_1,three_point_attempts_bin_2,three_point_attempts_bin_3,three_point_attempts_bin_4,three_point_percentage_bin_1,three_point_percentage_bin_2,three_point_percentage_bin_3,three_point_percentage_bin_4,three_pointers_bin_1,three_pointers_bin_2,three_pointers_bin_3,three_pointers_bin_4,total_rebounds_bin_1,total_rebounds_bin_2,total_rebounds_bin_3,total_rebounds_bin_4,turnovers_bin_1,turnovers_bin_2,turnovers_bin_3,turnovers_bin_4,two_point_attempts_bin_1,two_point_attempts_bin_2,two_point_attempts_bin_3,two_point_attempts_bin_4,two_point_percentage_bin_1,two_point_percentage_bin_2,two_point_percentage_bin_3,two_point_percentage_bin_4,two_pointers_bin_1,two_pointers_bin_2,two_pointers_bin_3,two_pointers_bin_4,seasons_played_bin_2,seasons_played_bin_3,seasons_played_bin_4,assist_percentage_bin_1,assist_percentage_bin_2,assist_percentage_bin_3,assist_percentage_bin_4,assist_percentage_bin_5,turnover_percentage_bin_1,turnover_percentage_bin_2,turnover_percentage_bin_3,turnover_percentage_bin_4,turnover_percentage_bin_5,usage_percentage_bin_1,usage_percentage_bin_2,usage_percentage_bin_3,usage_percentage_bin_4,usage_percentage_bin_5,usage_percentage_bin_6,defensive_win_shares_bin_1,defensive_win_shares_bin_2,defensive_win_shares_bin_3,defensive_win_shares_bin_4,offensive_win_shares_bin_1,offensive_win_shares_bin_2,offensive_win_shares_bin_3,height_to_weight_bin_1,height_to_weight_bin_2,height_to_weight_bin_3,height_to_weight_bin_5,weight_to_height_bin_1,weight_to_height_bin_2,weight_to_height_bin_3,weight_to_height_bin_4,weight_to_height_bin_5,recruit_draft_rank_mean_bin_1,recruit_draft_rank_mean_bin_2,draft_recruit_ratio_bin_1,draft_recruit_ratio_bin_2,draft_recruit_ratio_bin_3,draft_recruit_ratio_bin_4,draft_recruit_ratio_bin_5,assist_to_turnover_ratio_bin_1,assist_to_turnover_ratio_bin_2,assist_to_turnover_ratio_bin_3,assist_to_turnover_ratio_bin_4,assist_to_turnover_ratio_bin_5,assist_to_turnover_ratio_bin_6,ass

Select columns that are common to both the historical and active player datasets so we can perform comparisons.

In [7]:
cols = list(set(binned.columns).intersection(set(active.columns)))

In [8]:
df = pd.concat([active[cols], binned[cols]])
df.drop_duplicates(inplace = True)
df = df.sort_values(by = 'active', ascending = False).drop_duplicates(subset = ['ncaa_id'])
df.drop(['active'], axis = 1, inplace = True)
print(df.shape)
df.head()

(23864, 394)


,turnover_percentage_bin_3,assists_per_minute_bin_6,total_win_shares,field_goal_percentage_bin_3,recruit_draft_rank_mean_bin_2,turnover_percentage_bin_2,stocks_per_minute,steals_per_minute_bin_6,turnovers_bin_2,win_share_diff_off_bin_1,fga_per_minute_bin_3,minutes_per_game,points_rebounds_assists_per_minute_bin_6,minutes_played,two_pointers_bin_3,free_throw_attempts_bin_3,fgm_per_minute_bin_4,win_share_diff_def_bin_3,turnovers_bin_4,height,assist_percentage_bin_2,free_throw_attempts_bin_4,three_to_two_attempt_ratio_bin_3,field_goals_bin_4,start_rate_bin_3,orb_per_minute_bin_2,assist_to_turnover_ratio_bin_6,points_bin_4,stocks_bin_4,points_rebounds_assists_bin_2,def_ws_per_minute,fouls_per_minute_bin_1,def_ws_per_minute_bin_3,off_ws_per_minute_bin_4,recruit_draft_rank_mean_bin_1,assists_bin_4,high_school_recruit_score_bin_3,win_share_diff_def_bin_5,drb_orb_ratio_bin_4,def_ws_per_minute_bin_4,assist_percentage,three_point_percentage,assists_rebounds_ratio_bin_2,free_throw_attempts_bin_1,points_rebounds_assists,three_pointers_bin_4,drb_per_minute_bin_2,drb_per_minute_bin_5,win_shares_per_minute_bin_6,points_assists_ratio_bin_3,defensive_rebounds_bin_1,fgm_per_minute_bin_3,field_goal_percentage_bin_1,weight_bin_3,two_point_attempts_bin_1,minutes_played_bin_2,threes_made_per_minute_bin_4,fouls_per_minute_bin_2,weight_to_height_bin_2,games_played_bin_2,three_to_two_make_ratio,blocks_per_minute_bin_3,win_shares_per_minute_bin_2,total_win_shares_bin_1,orb_per_minute,three_attempts_per_minute_bin_1,two_point_attempts_bin_3,assist_percentage_bin_4,steals_bin_4,steals_bin_2,stocks_bin_2,steals_per_minute_bin_5,blocks,usage_percentage_bin_1,three_point_attempt_rate_bin_3,fgm_per_minute_bin_2,points_per_minute,def_ws_per_minute_bin_0,free_throw_attempt_rate_bin_6,offensive_rebounds_bin_4,three_point_attempts_bin_3,defensive_rebounds_bin_4,assists_bin_2,height_bin_2,def_ws_per_minute_bin_2,minutes_played_bin_3,assists_rebounds_ratio_bin_1,drb_orb_ratio_bin_2,assists_per_minute_bin_2,two_pointers_bin_2,assists,drb_per_minute_bin_3,win_share_diff_off_bin_6,height_to_weight_bin_2,assists_per_minute_bin_1,off_ws_per_minute_bin_5,offensive_win_shares,free_throw_attempt_rate_bin_2,blocks_bin_3,three_to_two_make_ratio_bin_2,defensive_win_shares_bin_2,steals_per_minute_bin_1,stocks_per_minute_bin_3,personal_fouls,start_rate_bin_2,field_goal_percentage_bin_4,offensive_rebounds_bin_1,steals,turnover_percentage,recruit_draft_rank_mean,games_started,fouls_per_minute_bin_4,three_attempts_per_minute_bin_2,points_rebounds_ratio_bin_3,drb_orb_ratio_bin_5,turnovers_bin_3,win_share_diff_def_bin_4,blocks_per_minute_bin_5,threes_made_per_minute_bin_6,points,win_share_diff_off,two_point_percentage_bin_4,two_point_attempts_bin_2,points_rebounds_ratio_bin_1,drb_per_minute,height_bin_5,field_goal_attempts_bin_3,three_to_two_attempt_ratio_bin_4,free_throw_percentage_bin_2,three_point_percentage_bin_1,two_point_percentage_bin_3,fgm_per_minute_bin_6,three_point_percentage_bin_3,points_per_minute_bin_4,weight_bin_1,points_rebounds_assists_per_minute_bin_1,off_ws_per_minute_bin_8,drb_orb_ratio_bin_3,three_to_two_attempt_ratio,defensive_rebounds,fouls_per_minute_bin_6,fga_per_minute,high_school_recruit_score_bin_2,height_bin_1,offensive_win_shares_bin_3,field_goal_attempts_bin_2,player_name,turnovers_bin_1,points_bin_2,two_pointers_bin_1,points_rebounds_ratio_bin_5,games_started_bin_2,weight_bin_4,three_point_attempt_rate_bin_2,personal_fouls_bin_2,three_point_attempt_rate_bin_1,field_goal_attempts_bin_4,offensive_rebounds_bin_3,free_throw_attempts,field_goal_percentage,draft_rank_bin_5,drb_orb_ratio_bin_6,points_rebounds_assists_bin_4,minutes_played_bin_4,start_rate_bin_1,turnovers,free_throw_attempt_rate,assist_to_turnover_ratio_bin_2,points_rebounds_assists_bin_1,win_share_diff_def,points_bin_1,fouls_per_minute_bin_3,steals_per_minute,points_assists_ratio_bin_4,total_rebounds_bin_1,points_assists_ratio_bin_5,personal_fouls_bin_3,usage_percentage_bin_2,high_school_nat

In [9]:
features = df.select_dtypes(['float', 'int']).columns.to_list()

In [10]:
top_60 = data.ncaa_id.unique()

In [11]:
nba_comps = df.loc[df.made_nba == 1].reset_index(drop = True)
nba_ids = nba_comps.ncaa_id.unique()

For each 2020 NBA draft prospect, generate three NBA comparisons based on binned data comparisons. The top three matches are calculated based on euclidean distance.

In [12]:
data['comparison_1'] = None
data['comparison_2'] = None
data['comparison_3'] = None

for p in top_60:
    comp = {}
    x = df.loc[df.ncaa_id == p][features]
    for n in nba_ids:
        y = nba_comps.loc[nba_comps.ncaa_id == n][features]
        comp.update({n : euclidean(x, y)})
    comps = sorted(comp, key = comp.get, reverse = False)[:3]
    data.loc[data.ncaa_id == p, 'comparison_1'] = comps[0]
    data.loc[data.ncaa_id == p, 'comparison_2'] = comps[1]
    data.loc[data.ncaa_id == p, 'comparison_3'] = comps[2]

Define our `draft_data_2020` table schema.

In [14]:
cursor = conn.cursor()
c_table = []
for c in data.columns.tolist():
    if data[c].dtype in [float, int, 'int64']:
        c_table.append('{} numeric'.format(c))
    elif data[c].dtype == 'datetime64[ns]':
        c_table.append('{} TIMESTAMP'.format(c))
    else:
        c_table.append('{} varchar({})'.format(c, max([len(str(x)) for x in data[c] if x is not None])))
create_table = 'DROP TABLE IF EXISTS draft_data_2020; '
create_table += 'CREATE TABLE draft_data_2020 ({})'.format(', '.join(c for c in c_table))
cursor.execute(create_table)
conn.commit()

Insert our player comparisons data into the `draft_data_2020` table.

In [15]:
draft_df = data.where(pd.notnull(data), None)
cursor = conn.cursor()
INSERT_SQL = 'INSERT INTO draft_data_2020'
INSERT_SQL += '({}) VALUES'.format(', '.join([x for x in draft_df.columns]))
INSERT_SQL += '({})'.format(''.join(['%s,' * len(draft_df.columns)])[:-1])
with conn, conn.cursor() as cursor:
    for row in draft_df.itertuples(index=False, name=None):
        cursor.execute(INSERT_SQL, row)